In [ ]:
# Importing libraries for data manipulation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Importing libraries for machine learning
from joblib import dump,load
import shap

# Display setting for exploration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# Check out if the environment is the correct Anaconda one
import sys
print('environment: ',sys.executable)

# Set up directory to be the github repository
# requires git
import os
import subprocess
os.getcwd()
output = subprocess.check_output(['git', 'rev-parse', '--show-toplevel'])
path = output.decode('utf-8').strip()
print('working directory: ',path)
os.chdir(path)


In [ ]:
from Source.DataPreparation.DataProcessor import DataProcessor
from Source.Classification import GBoostClassification, LogisticRegressionClass

In [ ]:
# Assuming you have data in X and y variables
Results_RF = []
Results_XGB = []
Results_LM=[]

montecarloiter=10

for i in range(montecarloiter):
    print(f"Starting the iteration n.{i+1}")

    processorclass=DataProcessor()
    #processorclass.read_df()
    #processorclass.split_data(test_size=0.3)
    #processorclass.save_data()
    #processorclass.process_CFA()

    X_train,y_train,X_test,y_test = processorclass.train_test_data_for_WEtarget(target_variable='WorkEngagement',Categories=False)

    # XGBRegressor
    Xgboost=GBoostRegression(x_train=X_train,y_train=y_train,x_test=X_test,y_test=y_test)
    Xgboost.train(verbosity=0,n_iter=100)
    mxgbresults = Xgboost.get_results()
    print(mxgbresults['best_params'])
    mse_xgb, r2_xgb, mae_xgb, train_mse_xgb, train_r2_xgb = (mxgbresults['mse'],mxgbresults['r2'],mxgbresults['mae'],mxgbresults['train_mse'],mxgbresults['train_r2'])
    Results_XGB.append((mse_xgb, r2_xgb, mae_xgb,train_mse_xgb, train_r2_xgb))

    print(f"Iteration {i+1} XGBoost: MSE: {mse_xgb}, R^2: {r2_xgb}, MAE: {mae_xgb}, TRAIN_MSE:{train_mse_xgb}, TRAIN_R^2:{train_r2_xgb}" )

    # Linear Regression
    LMreg= LinearRegressionModel(x_train=X_train,y_train=y_train,x_test=X_test,y_test=y_test)
    LMreg.median_imputation()
    LMreg.train(computeshap=True)
    LMresults = LMreg.get_results()
    mse_LM, r2_LM, mae_LM, train_mse_LM, train_r2_LM  = (LMresults['mse'],LMresults['r2'],LMresults['mae'],LMresults['train_mse'],LMresults['train_r2'])
    Results_LM.append((mse_LM, r2_LM, mae_LM,train_mse_LM, train_r2_LM))

    print(f"Iteration {i+1} Linear Regression: MSE: {mse_LM}, R^2: {r2_LM}, MAE: {mae_LM}, TRAIN_MSE:{train_mse_LM}, TRAIN_R^2:{train_r2_LM}")

    print(("-------------------------------------------------"))

        self.results["train_accuracy"] = train_accuracy
        self.results["train_precision"] = train_precision
        self.results["train_recall"] = train_recall

In [ ]:
import numpy as np

def print_avg_metrics(Results, model_name):
    mse_avg = np.mean([res[0] for res in Results])
    r2_avg = np.mean([res[1] for res in Results])
    mae_avg = np.mean([res[2] for res in Results])
    train_mse_avg = np.mean([res[3] for res in Results])
    train_r2_avg = np.mean([res[4] for res in Results])
    
    print(f"Model: {model_name}")
    print(f"Average MSE over 10 iterations: {mse_avg}")
    print(f"Average MAE over 10 iterations: {mae_avg}")
    print(f"Average R^2 over 10 iterations: {r2_avg}")
    print(f"Average TRAIN MSE over 10 iterations: {train_mse_avg}")
    print(f"Average TRAIN R^2 over 10 iterations: {train_r2_avg}")
    print("-------------------------------------------------")

# Print the metrics for each model
print_avg_metrics(Results_XGB, "XGBoost")
print_avg_metrics(Results_RF, "Random Forest")
print_avg_metrics(Results_LM, "Linear Model")